### 1. Value Iteration





#### 1(a). State Value Computation using Bellman Optimality Equation


In [653]:
def compute_state_value(state, V, transition_probs, rewards, actions, gamma, states):
    """
    Computes the value of a state using the Bellman optimality equation.

    Parameters:
    - state: Current state.
    - V: Current value function estimate.
    - transition_probs: Transition model of the environment (P(s' | s, a)).
    - rewards: Reward function (R(s, a, s')).
    - actions: Actions available in the state.
    - gamma: Discount factor for future rewards.
    - states: List of all states.

    Returns:
    - Value of the state (V*(s)).
    """

    # List to store expected values for each action.
    expected_values = []

    # Iterate over each action available in the current state.
    for action in actions[state]:

        # Initialize the expected value for this action (corresponding to the summation term in the Bellman equation).
        action_value = 0

        # Calculate the expected value for the action by summing over all possible next states.
        for next_state in states:

            # Transition probability from 'state' to 'next_state' given 'action' (P(s' | s, a)).
            transition_prob = transition_probs[state][action][next_state]

            # Immediate reward for taking 'action' in 'state' and ending up in 'next_state'
            immediate_reward = rewards[state][action][next_state]

            # Update the action's expected value using the Bellman equation components.
            action_value += transition_prob * (immediate_reward + gamma * V[next_state])

        # Append the expected value of the action to the list.
        expected_values.append(action_value)

    # Return the highest expected value among all actions, which corresponds to the max_a operation in the Bellman equation.
    return max(expected_values)




#### 1(b). Policy Extraction based on Value Function



In [654]:
def extract_policy(V, transition_probs, rewards, actions, gamma, states):
    """
    Extracts the optimal policy based on a given value function using the Bellman optimality equation.

    Parameters:
    - V: Value function based on which policy is extracted.
    - transition_probs: Transition model of the environment (P(s' | s, a)).
    - rewards: Reward function (R(s, a, s')).
    - actions: Dictionary of available actions for each state.
    - gamma: Discount factor for future rewards.
    - states: List of all states.

    Returns:
    - Optimal policy (a mapping from states to optimal actions).
    """

    policy = {}

    # Construct the policy by iterating over each state.
    for state in states:

        best_action_value = float('-inf')
        best_action = None

        # Evaluate each possible action to find the best one for this state
        for action in actions[state]:

            action_value = 0  # Initialize the expected value for this action

            # Compute the expected value for the state using this action
            for next_state in states:
                transition_prob = transition_probs[state][action][next_state]

                # Immediate reward for taking 'action' in 'state'.
                immediate_reward = rewards[state][action][next_state]

                # Update the action's value using components of the Bellman equation
                action_value += transition_prob * (immediate_reward + gamma * V[next_state])

            # Update the best action if this action's value is higher
            if action_value > best_action_value:
                best_action_value = action_value
                best_action = action

        # Update the policy for the current state with the best action
        policy[state] = best_action

    return policy


#### 1(c). Value Iteration Algorithm



In [655]:
def value_iteration(states, actions, transition_probs, rewards, gamma=0.9, theta=1e-3, debug_print=False):
    """
    Implements the value iteration algorithm to find the optimal policy and value function.

    Parameters:
    - states: List of states.
    - actions: Dictionary of actions.
    - transition_probs: Transition probabilities in a 3-D dictionary format.
    - rewards: Reward dictionary.
    - gamma: Discount factor for future rewards.
    - theta: Convergence threshold. Value iteration stops when the change in value function is smaller than eps.

    Returns:
    - V: Optimal value function.
    - policy: Optimal policy.
    """

    # Initialization
    V = {state: 0 for state in states}
    iteration = 0

    while True:
        new_V = V.copy()
        delta = 0

        # Value Update
        for state in states:
            new_V[state] = compute_state_value(state, V, transition_probs, rewards, actions, gamma, states)
            delta = max(delta, abs(new_V[state] - V[state]))

        # Optional: Print debug information
        if debug_print:
            iteration += 1
            print(f"Value Iteration {iteration}: Max Change: {delta}, Value Function: {new_V}")

        # Convergence Check
        if delta < theta:
            break

        V = new_V

    # Policy Extraction
    policy = extract_policy(V, transition_probs, rewards, actions, gamma, states)

    return V, policy


### 2. Policy Iteration


#### 2(a). Policy Evaluation



*

In [656]:
def policy_evaluation(policy, transition_probs, rewards, gamma, theta, states):
    """
    Evaluate the value function of a given policy using the Bellman expectation equation.

    Parameters:
    - policy: The policy to be evaluated.
    - transition_probs: Transition model of the environment (p(s', r | s, a)).
    - rewards: Reward function.
    - gamma: Discount factor for future rewards.
    - eps: Convergence threshold. Stops evaluation once value function changes are smaller than eps.
    - states: List of all states.

    Returns:
    - V: Value function of the given policy (v_pi).
    """

    # Initialization: Start with an arbitrary value function
    V = {state: 0 for state in states} # Dictionary

    while True:
        new_V = V.copy()

        # Update each state's value based on the Bellman expectation equation
        for state in states: # Select a state from list of all states
            # Given the deterministic policy, we directly get the action for the state
            action = policy[state] # Following the given policy select the next action for the state

            # Initialize the state's value to 0 and delta to 0 for this iteration
            delta = 0
            state_value=0

            # Compute the expected value for the state using its policy action
            for next_state in states:
                transition_prob = transition_probs[state][action][next_state]
                reward = rewards[state][action][next_state]
                # Update the state value for the action given the policy
                state_value += transition_prob * (reward + gamma * V[next_state])

            # Assign the computed state value to the new value function
            new_V[state] = state_value

            # Convergence Check
            delta = max(delta, abs(new_V[state] - V[state]))

        if delta < theta:
            break

        # Update the value function for the next iteration
        V = new_V

    return V


#### 2(b). Policy Improvement

In [657]:
def policy_improvement(V, transition_probs, rewards, actions, gamma, states):
    """
    Improve the policy based on a given value function using the Bellman optimality equation.

    Parameters:
    - V: Current estimate of the value function.
    - transition_probs: Transition model of the environment.
    - rewards: Reward function.
    - actions: Dictionary of available actions for each state.
    - gamma: Discount factor for future rewards.
    - states: List of all states.

    Returns:
    - new_policy: Improved policy.
    """

    new_policy = {}

    # Iterate through each state to update its policy based on action
    for state in states:

        # Initialize the best action and its corresponding value for this state
        best_action = None
        best_value = float('-inf')

        # Evaluate each possible action to find the best one for this state
        for action in actions[state]:
            action_value = 0

            # Calculate the expected value of this action over all next possible states
            for next_state in states:
                transition_prob = transition_probs[state][action][next_state]
                reward = rewards[state][action][next_state]
                action_value += transition_prob * (reward + gamma * V[next_state])

            # Update best action if this action's value is higher
            if action_value > best_value:
                best_value = action_value
                best_action = action

        # Assign the best action to the policy for this state
        new_policy[state] = best_action

    return new_policy


#### 2(c). Policy Iteration Algorithm

In [658]:
def policy_iteration(states, actions, transition_probs, rewards, gamma=0.9, theta=1e-3, debug_print=False):
    """
    Implements the policy iteration algorithm to find the optimal policy and value function.

    Parameters:
    - states: List of states.
    - actions: Dictionary of available actions for each state.
    - transition_probs: Transition model of the environment.
    - rewards: Reward dictionary.
    - gamma: Discount factor for future rewards.
    - theta: Convergence threshold for policy evaluation phase.

    Returns:
    - V: Optimal value function.
    - policy: Optimal policy.
    """

    # Initialization: Arbitrarily choose an initial policy
    # Here, we initialize the policy for each state as the first available action.
    policy = {state: actions[state][0] for state in states}
    iteration_count = 0

    while True:
        iteration_count += 1

        # Policy Evaluation: Compute the state-value function for the current policy
        V = policy_evaluation(policy, transition_probs, rewards, gamma, theta, states)

        if debug_print:
            print(f"Policy Iteration {iteration_count}: Value Function after Policy Evaluation: {V}")

        # Policy Improvement: Update the policy to be greedy with respect to the current value function
        new_policy = policy_improvement(V, transition_probs, rewards, actions, gamma, states)

        if debug_print:
            print(f"Policy Iteration {iteration_count}: Policy after Improvement: {new_policy} \n")

        # Convergence Check: If the policy remains unchanged after the improvement step, it's optimal
        if new_policy == policy:
            break

        # Update the policy for the next iteration
        policy = new_policy

    return V, policy


## 3. MDP Implementation


This time we generalise the MDP structure with solving functions, so that we can use any given scenerio. There are possibilities that depending on the scenerio, we modify the functions.

In [659]:
import numpy as np
import time

class MDP:
    def __init__(self, states, actions, transition_matrix, reward_matrix, discount_factor=1.0):
        """
        Initialize the MDP with given states, actions, transition probabilities, rewards, and discount factor.

        Parameters:
        - states: List of states in the MDP
        - actions: List of actions available in the MDP
        - transition_matrix: Matrix where each row represents the current state, each column represents an action,
                             and the inner lists represent the next state probabilities.
        - reward_matrix: Matrix where each row represents the current state and each column represents an action.
        - discount_factor: Discount factor for future rewards (gamma in Sutton & Barto)
        """
        self.states = states
        self.actions = actions
        self.transition_matrix = transition_matrix
        self.reward_matrix = reward_matrix
        self.discount_factor = discount_factor

    # Converting the Transition Prob, rewards and actions to Dictionary.
    def convert_to_dictionary(self):
        """
        Convert transition matrix and reward matrix to a dictionary format which is more intuitive for certain operations.

        Returns:
        - transition_probs: Dictionary of transition probabilities
        - rewards: Dictionary of rewards for state-action pairs
        - actions: Dictionary of available actions for each state
        """
        # Convert actions list to dictionary format
        actions = {state: [act for act in self.actions] for state in self.states}

        # Initialize the transition_probs and rewards dictionaries
        transition_probs = {s: {} for s in self.states}
        rewards = {s: {} for s in self.states}

        for i, s in enumerate(self.states):
            for j, a in enumerate(self.actions):
                transition_probs[s][a] = {}
                rewards[s][a] = {}
                for k, s_prime in enumerate(self.states):
                    # Set the transition probability for s' from the matrix
                    transition_probs[s][a][s_prime] = self.transition_matrix[i][j][k]

                    # Set the reward for action a in state s from the matrix
                    rewards[s][a][s_prime] = self.reward_matrix[i][j][k]

        return transition_probs, rewards, actions



### Define Specifics
* States
* Actions
* Transition Probabilities (Matrix)
* Rewards (Matrix)

In [660]:
# Define states

states = ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
# Define the actions
actions = ["Go Fast", "Keep Medium Speed", "Go Slow", "Stop","Change Lane","Steer Around"]


# Define transition matrices for each state,actio,next state
transition_matrix = [
    [
        # From "Pedestrian"
        [1, 0, 0, 0, 0, 0],  # Go Fast to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [1, 0, 0, 0, 0, 0],  # Keep Medium Speed to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [1, 0, 0, 0, 0, 0],  # Go Slow to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [0, 0, 0, 1, 0, 0],  # Stop to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [1, 0, 0, 0, 0, 0],  # Change Lane to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [1, 0, 0, 0, 0, 0]  # Steer Around to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
    ],
    [
        # From "Clear Road"
        [0, 0.1, 0.3, 0.2, 0.2, 0.2],  # Go Fast to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [0, 0.4, 0.2, 0.2, 0.2, 0],  # Keep Medium Speed to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [1, 0, 0, 0, 0, 0],  # Go Slow to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [1, 0, 0, 0, 0, 0],  # Stop to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [1, 0, 0, 0, 0, 0],  # Change Lane to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [1, 0, 0, 0, 0, 0.6]  # Steer Around to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
    ],
    [
        # From "Destination"
        [0, 0, 1, 0, 0, 0],  # Go Fast to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [0, 0, 1, 0, 0, 0],  # Keep Medium Speed to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [0, 0, 1, 0, 0, 0],  # Go Slow to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [0, 0, 1, 0, 0, 0],  # Stop to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [0, 0, 1, 0, 0, 0],  # Change Lane to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [0, 0, 1, 0, 0, 0]  # Steer Around to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
    ],
    [
        # From "Crowded Area"
        [1, 0, 0, 0, 0, 0],  # Go Fast to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [0.2, 0.5, 0, 0.3, 0, 0],  # Keep Medium Speed to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [0.3, 0.3, 0, 0.4, 0, 0],  # Go Slow to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [1, 0, 0, 0, 0, 0],  # Stop to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [1, 0, 0, 0, 0, 0],  # Change Lane to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [1, 0, 0, 0, 0, 0] # Steer Around to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
    ],
    [
        # From "Obstacle"
        [1, 0, 0, 0, 0, 0],  # Go Fast to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [1, 0, 0, 0, 0, 0],  # Keep Medium Speed to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [1, 0, 0, 0, 0, 0],  # Go Slow to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [0, 1, 0, 0, 0, 0],  # Stop to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [0, 0.8, 0, 0, 0.2, 0],  # Change Lane to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [1, 0.6, 0, 0, 0.4, 0]  # Steer Around to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
    ],
    [
        #From "Slow Car"
        [1, 0, 0, 0, 0, 0],  # Go Fast to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [1, 0, 0, 0, 0, 0],  # Keep Medium Speed to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [0, 0.4, 0, 0, 0, 0.6],  # Go Slow to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
        [1, 0, 0, 0, 0, 0],  # Stop to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
       [0, 0.8, 0, 0, 0.1, 0.1], # Change Lane to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
         [1, 0, 0, 0, 0, 0]  # Steer Around to ["Pedestrian", "Clear Road", "Destination", "Crowded Area", "Obstacle", "Slow Car"]
    ]
]




# Rewards structure: Dictionary to hold a matrix for each starting state
import numpy as np

reward_matrix = [
    [
        [-1000, -1000, -1000, -1000, -1000, -1000],  # Go Fast (Pedestrian)
        [-1000, -1000, -1000, -1000, -1000, -1000],  # Keep Medium Speed (Pedestrian)
        [-1000, -1000, -1000, -1000, -1000, -1000],  # Go Slow (Pedestrian)
        [-10, -1000, -100, -1000, -1000, -1000],    # Stop (Pedestrian)
        [-1000, -1000, -1000, -1000, -1000, -1000],  # Change Lane (Pedestrian)
        [-1000, -1000, -1000, -1000, -1000, -1000]   # Steer Around (Pedestrian)
    ],
    [
        [-1000, 4, 100, -10, -15, -10],             # Go Fast (Clear Road)
        [-1000, 2, 100, -2, -5, -1000],              # Keep Medium Speed (Clear Road)
        [-1000, -1000, -1000, -1000, -1000, -1000], # Go Slow (Clear Road)
        [-1000, -1000, -1000, -1000, -1000, -1000], # Stop (Clear Road)
        [-1000, -1000, -1000, -1000, -1000, -1000], # Change Lane (Clear Road)
        [-1000, -1000, -1000, -1000, -1000, -1000]  # Steer Around (Clear Road)
    ],
    [   [-1000, -1000, -1000, -1000, -1000, -1000],     # Go Fast (Destination)
        [-1000, -1000, -1000, -1000, -1000, -1000],     # Keep Medium Speed (Destination)
        [-1000, -1000, -1000, -1000, -1000, -1000],     # Go Slow (Destination)
        [-1000, -1000,0, -1000, -1000, -1000],     # Stop (Destination)
        [-1000, -1000, -1000, -1000, -1000, -1000],     # Change Lane (Destination)
        [-1000, -1000, -1000, -1000, -1000, -1000]      # Steer Around (Destination)

    ],
    [   [-1000, -1000, -1000, -1000, -1000, -1000], # Go Fast (Crowded Area)
        [-7, 2, -1000, -1000, -4, -1000],       # Keep Medium Speed (Crowded Area)
        [-2, 2, -1000,-1000, -2, -1000],        # Go Slow (Crowded Area)
        [-1000, -1000, -1000, -1000, -1000, -1000], # Stop (Crowded Area)
        [-1000, -1000, -1000, -1000, -1000, -1000], # Change Lane (Crowded Area)
        [-1000, -1000, -1000, -1000, -1000, -1000]  # Steer Around (Crowded Area)
    ],
    [
        [-1000, -1000, -1000, -1000, -1000, -1000], # Go Fast (Obstacle)
        [-1000, -1000, -1000, -1000, -1000, -1000], # Keep Medium Speed (Obstacle)
        [-1000, -1000, -1000, -1000, -1000, -1000], # Go Slow (Obstacle)
        [-1000, -25, -1000, -1000, -1000,-1000],    # Stop (Obstacle)
        [-1000, 3, -1000, -1000, -3, -1000],        # Change Lane (Obstacle)
        [-1000, 5, -1000, -1000, -5, -1000]         # Steer Around (Obstacle)
    ],
    [
        [-1000, -1000, -1000, -1000, -1000, -1000], # Go Fast (Slow Car)
        [-1000, -1000, -1000, -1000, -1000, -1000], # Keep Medium Speed (Slow Car)
        [-1000, 2, -1000, -1000, -1000, -2],        # Go Slow (Slow Car)
        [-1000, -1000, -1000, -1000, -1000, -1000], # Stop (Slow Car)
        [-1000, 4, -1000, -1000, -5, -5],           # Change Lane (Slow Car)
        [-1000, -1000, -1000, -1000, -1000, -1000]  # Steer Around (Slow Car)
    ]
]



### Create an instance of the MDP with the defined specifics
* States
* Actions
* Transition Probabilities (Matrix)
* Rewards (Matrix)

In [661]:
# Create an MDP instance
car_MDP = MDP(states, actions, transition_matrix, reward_matrix)

In [662]:
# Convert to Dictionary before proceeding with Policy Iteration
transition_probs, rewards, actions = car_MDP.convert_to_dictionary()
print(transition_probs)

{'Pedestrian': {'Go Fast': {'Pedestrian': 1, 'Clear Road': 0, 'Destination': 0, 'Crowded Area': 0, 'Obstacle': 0, 'Slow Car': 0}, 'Keep Medium Speed': {'Pedestrian': 1, 'Clear Road': 0, 'Destination': 0, 'Crowded Area': 0, 'Obstacle': 0, 'Slow Car': 0}, 'Go Slow': {'Pedestrian': 1, 'Clear Road': 0, 'Destination': 0, 'Crowded Area': 0, 'Obstacle': 0, 'Slow Car': 0}, 'Stop': {'Pedestrian': 0, 'Clear Road': 0, 'Destination': 0, 'Crowded Area': 1, 'Obstacle': 0, 'Slow Car': 0}, 'Change Lane': {'Pedestrian': 1, 'Clear Road': 0, 'Destination': 0, 'Crowded Area': 0, 'Obstacle': 0, 'Slow Car': 0}, 'Steer Around': {'Pedestrian': 1, 'Clear Road': 0, 'Destination': 0, 'Crowded Area': 0, 'Obstacle': 0, 'Slow Car': 0}}, 'Clear Road': {'Go Fast': {'Pedestrian': 0, 'Clear Road': 0.1, 'Destination': 0.3, 'Crowded Area': 0.2, 'Obstacle': 0.2, 'Slow Car': 0.2}, 'Keep Medium Speed': {'Pedestrian': 0, 'Clear Road': 0.4, 'Destination': 0.2, 'Crowded Area': 0.2, 'Obstacle': 0.2, 'Slow Car': 0}, 'Go Slow': {

### Solve MDP using Value Iteration:

In [663]:
v, p = value_iteration(car_MDP.states, actions, transition_probs, rewards, debug_print=True)
print("Value Iteration: Value function: ", v)
print("Value Iteration: Policy: ", p)

Value Iteration 1: Max Change: 1000.0, Value Function: {'Pedestrian': -1000.0, 'Clear Road': 23.4, 'Destination': 0.0, 'Crowded Area': -300.4, 'Obstacle': 1.8000000000000003, 'Slow Car': 2.2}
Value Iteration 2: Max Change: 270.3600000000001, Value Function: {'Pedestrian': -1270.3600000000001, 'Clear Road': -25.924000000000007, 'Destination': 0.0, 'Crowded Area': -550.9780000000001, 'Obstacle': 18.972, 'Slow Car': 19.408}
Value Iteration 3: Max Change: 225.52019999999993, Value Function: {'Pedestrian': -1495.8802, 'Clear Road': -71.2008, 'Destination': 0.0, 'Crowded Area': -689.4946600000001, 'Obstacle': -13.450320000000003, 'Slow Car': 0.747679999999999}
Value Iteration 4: Max Change: 124.66499399999998, Value Function: {'Pedestrian': -1620.545194, 'Clear Road': -109.403586, 'Destination': 0.0, 'Crowded Area': -787.8623542, 'Obstacle': -51.8856336, 'Slow Car': -25.628540800000003}
Value Iteration 5: Max Change: 88.53092477999985, Value Function: {'Pedestrian': -1709.07611878, 'Clear Ro

### Solve MDP using Policy Iteration:

In [664]:
# Execute Policy Iteration Algorithm
V, policy = policy_iteration(car_MDP.states, actions, transition_probs, rewards, debug_print=True)

# Display outcomes
print("\n Policy Iteration: Value function: ", V)
print("Policy Iteration: Policy: ", policy)

Policy Iteration 1: Value Function after Policy Evaluation: {'Pedestrian': -9999.990879655443, 'Clear Road': -8875.375495040058, 'Destination': -9999.990879655443, 'Crowded Area': -9999.990879655443, 'Obstacle': -9999.990879655443, 'Slow Car': -9999.990879655443}
Policy Iteration 1: Policy after Improvement: {'Pedestrian': 'Go Fast', 'Clear Road': 'Keep Medium Speed', 'Destination': 'Stop', 'Crowded Area': 'Keep Medium Speed', 'Obstacle': 'Stop', 'Slow Car': 'Change Lane'} 

Policy Iteration 2: Value Function after Policy Evaluation: {'Pedestrian': -9999.955703072457, 'Clear Road': -1370.427567599078, 'Destination': 0.0, 'Crowded Area': -3722.0307715346476, 'Obstacle': -1258.3838264630028, 'Slow Car': -1206.3312183261967}
Policy Iteration 2: Policy after Improvement: {'Pedestrian': 'Stop', 'Clear Road': 'Go Fast', 'Destination': 'Stop', 'Crowded Area': 'Keep Medium Speed', 'Obstacle': 'Change Lane', 'Slow Car': 'Go Slow'} 

Policy Iteration 3: Value Function after Policy Evaluation: {'

### Compare both algorithms

In [665]:
def simulate_for_average_reward(states, transition_probs, rewards, start_state, policy, max_steps=100, num_episodes=1000):
    """
    Simulate the environment to compute the average reward over a number of episodes.

    Parameters:
    - states: List of states
    - transition_probs: Dictionary of transition probabilities
    - rewards: Dictionary of rewards for state-action pairs
    - start_state (Any): The initial state from which the simulation begins.
    - policy (dict): A mapping from states to actions representing the agent's policy.
    - max_steps (int, optional): Maximum number of steps for each episode. Defaults to 100.
    - num_episodes (int, optional): Number of episodes to simulate. Defaults to 1000.

    Returns:
    - float: The average reward accumulated per episode over the specified number of episodes.
    """
    total_rewards = 0
    for _ in range(num_episodes):
        current_state = start_state
        for _ in range(max_steps):
            action = policy[current_state]

            # Sample the next state based on the transition probabilities
            if np.sum(np.array(list(transition_probs[current_state][action].values())))!=0:
              # Convert transition_probs[current_state][action].values() to a list
              prob_list = list(transition_probs[current_state][action].values())
              # Normalize the prob_list to ensure it sums to 1
              prob_list = [p / sum(prob_list) for p in prob_list]
              next_state = np.random.choice(states, p=prob_list)

            # Access the reward using next_state
            reward = rewards[current_state][action][next_state]
            total_rewards += reward
            current_state = next_state
    return total_rewards / num_episodes

In [666]:
def compare_algorithms(states, actions, transition_probs, rewards, start_state, gamma=0.9, theta=1e-3):
    """
    Compare the performance of Policy Iteration (PI) and Value Iteration (VI) on the given environment.

    Parameters:
    - states: List of states
    - transition_probs: Dictionary of transition probabilities
    - rewards: Dictionary of rewards for state-action pairs
    - start_state (Any): The initial state from which the simulations begin.
    - gamma (float, optional): The discount factor used in both algorithms. Defaults to 0.9.
    - eps (float, optional): The threshold for determining convergence in both algorithms. Defaults to 1e-3.

    Returns:
    - dict: A dictionary containing performance metrics and results for both algorithms:
      * "Value Iteration" and "Policy Iteration" each contain a sub-dictionary with:
        - "Convergence Time": Time taken for the algorithm to converge.
        - "Value Function": The final value function determined by the algorithm.
        - "Policy": The optimal policy determined by the algorithm.
        - "Average Reward": The average reward when following the computed policy from the start state.
      * "Value Function Difference": The sum of absolute differences between value functions of PI and VI.
      * "Policy Difference": The number of states where PI and VI policies differ.

    Notes:
    This function first runs Policy Iteration and then Value Iteration on the environment. After both algorithms
    have been executed, it compares their value functions and policies, calculates the average reward for both
    policies, and compiles these results into a dictionary.
    """
    # Policy Iteration
    start_time = time.time()
    PI_values, PI_policy = policy_iteration(states, actions, transition_probs, rewards, gamma, theta, debug_print=False)
    PI_time = time.time() - start_time
    PI_avg_reward = simulate_for_average_reward(states, transition_probs, rewards, start_state, PI_policy)

    # Value Iteration
    start_time = time.time()
    VI_values, VI_policy = value_iteration(states, actions, transition_probs, rewards, gamma, theta, debug_print=False)
    VI_time = time.time() - start_time
    VI_avg_reward = simulate_for_average_reward(states, transition_probs, rewards, start_state, VI_policy)

    # Metrics
    value_diff = sum([abs(VI_values[state] - PI_values[state]) for state in states])
    policy_diff = sum([1 if VI_policy[state] != PI_policy[state] else 0 for state in states])

    return {
        "Value Iteration": {
            "Convergence Time": VI_time,
            "Value Function": VI_values,
            "Policy": VI_policy,
            "Average Reward": VI_avg_reward
        },
        "Policy Iteration": {
            "Convergence Time": PI_time,
            "Value Function": PI_values,
            "Policy": PI_policy,
            "Average Reward": PI_avg_reward
        },
        "Value Function Difference (Sum of Absolute Differences)": value_diff,
        "Policy Difference (Number of Different Actions)": policy_diff
    }


In [667]:
compare_algorithms(car_MDP.states, actions, transition_probs, rewards, 'Clear Road')

{'Value Iteration': {'Convergence Time': 0.0022459030151367188,
  'Value Function': {'Pedestrian': -1956.9254872930824,
   'Clear Road': -274.5061532402667,
   'Destination': 0.0,
   'Crowded Area': -1063.251497461386,
   'Obstacle': -238.8338865525919,
   'Slow Car': -215.69879510821937},
  'Policy': {'Pedestrian': 'Stop',
   'Clear Road': 'Go Fast',
   'Destination': 'Stop',
   'Crowded Area': 'Keep Medium Speed',
   'Obstacle': 'Change Lane',
   'Slow Car': 'Go Slow'},
  'Average Reward': -609.924},
 'Policy Iteration': {'Convergence Time': 0.0025110244750976562,
  'Value Function': {'Pedestrian': -1956.9244753595422,
   'Clear Road': -274.5055031057693,
   'Destination': 0.0,
   'Crowded Area': -1063.2505994886947,
   'Obstacle': -238.83312990376226,
   'Slow Car': -215.69789020524786},
  'Policy': {'Pedestrian': 'Stop',
   'Clear Road': 'Go Fast',
   'Destination': 'Stop',
   'Crowded Area': 'Keep Medium Speed',
   'Obstacle': 'Change Lane',
   'Slow Car': 'Go Slow'},
  'Average R

# TASK 2 STOCK TRADING

In [668]:
# Define states

states2 = ["Initial Increase", "Sustained Increase", "Initial Decrease", "Sustained Decrease", "Initial Stability",'Sustained stability']
# Define the actions
actions2 = ["Buy","Sell","Hold"]


# Define transition matrices for each state,actio,next state
transition_matrix2 = [
    # From Initial Increase to ["Initial Increase", "Sustained Increase", "Initial Decrease", "Sustained Decrease", "Initial Stability",'Sustained stability']
    [
        [0, 0.6, 0.2, 0, 0.2,0],  # Buy
        [0, 0.6, 0.2, 0, 0.2,0],  # Sell
        [0, 0.6, 0.2, 0, 0.2,0],  # Hold
    ],
    # From Sustained Increase
    [
        [0, 0.2, 0.4, 0, 0.4,0],  # Buy
        [0, 0.2, 0.4, 0, 0.4,0],  # Sell
        [0, 0.2, 0.4, 0, 0.4,0],  # Hold
    ],
    # From Initial Decrease
    [
        [0.2, 0, 0, 0.6, 0.2,0],  # Buy
        [0.2, 0, 0, 0.6, 0.2,0],  # Sell
        [0.2, 0, 0, 0.6, 0.2,0],  # Hold
    ],
    # From Sustained Decrease
    [
        [0.4, 0, 0, 0.2, 0.4,0],  # Buy
        [0.4, 0, 0, 0.2, 0.4,0],  # Sell
        [0.4, 0, 0, 0.2, 0.4,0],  # Hold
    ],
    # From Stability
    [
        [0.4, 0, 0.4, 0, 0,0.2],  # Buy
        [0.4, 0, 0.4, 0, 0,0.2],  # Sell
        [0.4, 0, 0.4, 0, 0,0.2],  # ,
            # From Sustained Stability
    ],[
        [0.1, 0, 0.1, 0, 0, 0.8],  # Buy
        [0.1, 0, 0.1, 0, 0, 0.8],  # Sell
        [0.1, 0, 0.1, 0, 0, 0.8]        # Hold
    ]
]


reward_matrix2 = [
    # From Initial Increase
    [
        [-1000, +10, -10, -1000, -1000, -1000],  # Buy
        [-1000, -10, +10, -1000, -1000, -1000],  # Sell
        [-1000, -1000, -1000, -1000, -1000, -1000],        # Hold
    ],
    # From Sustained Increase
    [
        [-1000, +10, -10, -1000, -1000, -1000],  # Buy
        [-1000, -10, +10, -1000, -1000, -1000],  # Sell
        [-1000, -1000, -1000, -1000, -1000, -1000],        # Hold
    ],
    # From Initial Decrease
    [
        [+10, -1000, -1000, -10, -1000, -1000],  # Buy
        [-10, -1000, -1000, +10, -1000, -1000],  # Sell
        [-1000, -1000, -1000, -1000, -1000, -1000],        # Hold
    ],
    # From Sustained Decrease
    [
        [+10, -1000, -1000, -10, -1000, -1000],  # Buy
        [-10, -1000, -1000, +10, -1000, -1000],  # Sell
        [-1000, -1000, -1000, -1000, -1000, -1000],        # Hold
    ],
    # From Initial Stability
    [
        [+10, -1000, -1000, -10, -1000, -1000],  # Buy
        [-10, -1000, -1000, +10, -1000, -1000],  # Sell
        [-1000, -1000, -1000, -1000, -1000, -1000],        # Hold
    ],
    # From Sustained Stability
    [
        [+10, -1000, -1000, -10, -1000, -1000],  # Buy
        [-10, -1000, -1000, +10, -1000, -1000],  # Sell
        [-1000, -1000, -1000, -1000, -1000, -1000],        # Hold
    ]
]




In [669]:
# Create an MDP instance
fin_MDP = MDP(states2, actions2, transition_matrix2, reward_matrix2)

In [670]:
# Convert to Dictionary before proceeding with Policy Iteration
transition_probs2, rewards2, actions2 = fin_MDP.convert_to_dictionary()
print(transition_probs2)

{'Initial Increase': {'Buy': {'Initial Increase': 0, 'Sustained Increase': 0.6, 'Initial Decrease': 0.2, 'Sustained Decrease': 0, 'Initial Stability': 0.2, 'Sustained stability': 0}, 'Sell': {'Initial Increase': 0, 'Sustained Increase': 0.6, 'Initial Decrease': 0.2, 'Sustained Decrease': 0, 'Initial Stability': 0.2, 'Sustained stability': 0}, 'Hold': {'Initial Increase': 0, 'Sustained Increase': 0.6, 'Initial Decrease': 0.2, 'Sustained Decrease': 0, 'Initial Stability': 0.2, 'Sustained stability': 0}}, 'Sustained Increase': {'Buy': {'Initial Increase': 0, 'Sustained Increase': 0.2, 'Initial Decrease': 0.4, 'Sustained Decrease': 0, 'Initial Stability': 0.4, 'Sustained stability': 0}, 'Sell': {'Initial Increase': 0, 'Sustained Increase': 0.2, 'Initial Decrease': 0.4, 'Sustained Decrease': 0, 'Initial Stability': 0.4, 'Sustained stability': 0}, 'Hold': {'Initial Increase': 0, 'Sustained Increase': 0.2, 'Initial Decrease': 0.4, 'Sustained Decrease': 0, 'Initial Stability': 0.4, 'Sustained 

In [671]:
v2, p2 = value_iteration(fin_MDP.states, actions2, transition_probs2, rewards2, debug_print=True)
print("Value Iteration: Value function: ", v2)
print("Value Iteration: Policy: ", p2)

Value Iteration 1: Max Change: 899.0, Value Function: {'Initial Increase': -196.0, 'Sustained Increase': -398.0, 'Initial Decrease': -196.0, 'Sustained Decrease': -398.0, 'Initial Stability': -596.0, 'Sustained stability': -899.0}
Value Iteration 2: Max Change: 682.56, Value Function: {'Initial Increase': -553.48, 'Sustained Increase': -754.76, 'Initial Decrease': -553.48, 'Sustained Decrease': -754.76, 'Initial Stability': -898.94, 'Sustained stability': -1581.56}
Value Iteration 3: Max Change: 555.7896000000001, Value Function: {'Initial Increase': -865.0060000000001, 'Sustained Increase': -1056.728, 'Initial Decrease': -865.0060000000001, 'Sustained Decrease': -1056.728, 'Initial Stability': -1279.1864, 'Sustained stability': -2137.3496}
Value Iteration 4: Max Change: 456.24319200000036, Value Function: {'Initial Increase': -1152.5877520000001, 'Sustained Increase': -1360.1203040000003, 'Initial Decrease': -1152.5877520000001, 'Sustained Decrease': -1360.1203040000003, 'Initial Stab

In [672]:
# Execute Policy Iteration Algorithm
V2, policy2 = policy_iteration(fin_MDP.states, actions2, transition_probs2, rewards2, debug_print=True)

# Display outcomes
print("\n Policy Iteration: Value function: ", V2)
print("Policy Iteration: Policy: ", policy2)

Policy Iteration 1: Value Function after Policy Evaluation: {'Initial Increase': -4064.3908648060337, 'Sustained Increase': -4283.5698372290535, 'Initial Decrease': -4068.1774396769456, 'Sustained Decrease': -4277.02938972475, 'Initial Stability': -4572.178748242382, 'Sustained stability': -5824.75066528691}
Policy Iteration 1: Policy after Improvement: {'Initial Increase': 'Buy', 'Sustained Increase': 'Sell', 'Initial Decrease': 'Sell', 'Sustained Decrease': 'Buy', 'Initial Stability': 'Buy', 'Sustained stability': 'Buy'} 

Policy Iteration 2: Value Function after Policy Evaluation: {'Initial Increase': -4043.59780496279, 'Sustained Increase': -4259.577140426999, 'Initial Decrease': -4043.59780496279, 'Sustained Decrease': -4259.577140426999, 'Initial Stability': -4553.219450692367, 'Sustained stability': -5810.166600027089}
Policy Iteration 2: Policy after Improvement: {'Initial Increase': 'Buy', 'Sustained Increase': 'Sell', 'Initial Decrease': 'Sell', 'Sustained Decrease': 'Buy', '

In [673]:
compare_algorithms(fin_MDP.states, actions2, transition_probs2, rewards2, 'Initial Stability',gamma=0.75)

{'Value Iteration': {'Convergence Time': 0.0010499954223632812,
  'Value Function': {'Initial Increase': -1422.4351484820472,
   'Sustained Increase': -1629.0692030445266,
   'Initial Decrease': -1422.4351484820472,
   'Sustained Decrease': -1629.0692030445266,
   'Initial Stability': -1866.5969546904364,
   'Sustained stability': -2780.911159768478},
  'Policy': {'Initial Increase': 'Buy',
   'Sustained Increase': 'Sell',
   'Initial Decrease': 'Sell',
   'Sustained Decrease': 'Buy',
   'Initial Stability': 'Buy',
   'Sustained stability': 'Buy'},
  'Average Reward': -44891.77},
 'Policy Iteration': {'Convergence Time': 0.0008389949798583984,
  'Value Function': {'Initial Increase': -1422.4351484820472,
   'Sustained Increase': -1629.0692030445266,
   'Initial Decrease': -1422.4351484820472,
   'Sustained Decrease': -1629.0692030445266,
   'Initial Stability': -1866.5969546904364,
   'Sustained stability': -2780.911159768478},
  'Policy': {'Initial Increase': 'Buy',
   'Sustained Incr